In [1]:
import numpy as np
import astropy.units as u
from astropy.constants import h,k_B, c, m_e
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.special import kn, iv

K_0 = lambda x: kn(0, x)
I_0 = lambda x: iv(0, x)

hbar = h/(2*np.pi)

I'll start by listing all the design variables of the resonator bolometer

1. C, L - Capacitance and Inductance of the ResoBolo
2. $V_{sc}$ - Volume of the Superconductor
3. $\alpha$ - Kinetic Inductance Fraction
4. $\gamma_s$ - surface index. Value of 0.5 for thick films, 1 for thin films
5. $Q_{int}$ - Internal Quality factor without qp losses
6. $f_g$ - readout frequency
7. $T_{amp}$ - Noise Temperature of the Amplifier
8. $\chi = P_{read}/P_g$ - Absorption efficiency of the readout power
9. $x = P_{read}/P_{opt}$ 
9. $\eta_{read}$ - quasiparticle generation efficiency by $P_{read}$, the readout power absorbed in the resonator

Optical Power on the Resonator Bolometer

1. $\nu_{opt}$ - Optical Frequency
2. $\Delta \nu_{opt}/\nu_{opt} $ - Optical Bandwidth
3. $\eta_{opt}$ - Optical Efficiency
4. $T_{rj}$ - Brightness Temperature
5. $N_{pol}$ - Polarizations Detected

Bolometer Island Parameters

1. $\gamma_{leg}$ - Conductivity Index
2. $K_{leg}$ - Conductivity of the Bolometer Leg
3. $T_c$ - Transition Temperature of the Superconductor
4. $T_0$ - Temperature of the Thermal Bath
5. $C_b$ - Heat Capacity of the Thermal Island

Surface Impedance Parameters

1. $\tau_{max}$ - Empirical Maximum Lifetime of Quasiparticles
2. $n_{qp}^*$ - quasiparticle density associated with the maximum lifetime of qps
3. $\gamma$ - heat capacity coefficient for Aluminium
4. $\rho$ - density of Aluminium
5. $A_r$ - atomic mass of Aluminium

Noise Parameters

1. $S_{TLS}$ - Two level system Noise

In [2]:
def pprinter(**kwargs):
    for arg in kwargs:
        print (arg  + " = {0:3.3g}".format(kwargs[arg]))

In [3]:
C = 70 * u.pF
L = 8.6 * u.nH
V_sc = 825 * u.micrometer**3
alpha = 0.4
gamma_s = 1
Q_int = 1e6
f_g = 205.128 * u.MHz
T_amp = 3 * u.Kelvin
x = 120.00
eta_read = 0.1

nu_opt = 270 * u.GHz
dnu_opt = 0.25
eta_opt = 0.25
T_rj = 0.1 * u.Kelvin
N_pol = 1

gamma_leg = 2.65
K_leg = 120 * u.picoWatt/u.Kelvin**gamma_leg
T_c = 1.2 * u.Kelvin
T_0 = 0.23 * u.Kelvin
C_b = 1 * u.picoJoule/u.Kelvin

tau_max = 500 * u.microsecond
n_qp_star = 1000 * 1/u.micrometer**3
gamma = 1.35 * u.mJ/u.mol/u.Kelvin**2
rho = 2.7 * u.g/u.cm**3
A_r = 26.98 * u.g/u.mol


Goal is to calculate the relevant parameters of the resonator. These parameters are listed below. But first we need to calculate several parameters of the bolometer to get a start on this.

We can calculate the optical power in the bolometer from the brightness temperature, $T_{rj}$. The brightness temperature is defined to be the temperature of a blackbody that has the same intensity as that measured at that frequency. Starting from $$I(\nu, T) = \frac{2 h \nu^3}{c^2} \frac{1}{\exp\left(h \nu/ k_B T\right) - 1}$$, we can make the approximation $h \nu \ll k_B T$ to give $I(\nu, T) = \frac{2 \nu^2}{c^2}{k_B T}$ as the definition of the brightness temperature. Now the total optical power is given by $$P_{opt} = \int \limits_{\nu_{min}}^{\nu_{max}} \mathrm d \nu \int \mathrm d A \int \mathrm d \Omega\ \ I(\nu, T) $$. Assuming the intensity is isotropic (no dependence on the solid angle) and with a small enough bandwidth, we arrive at the expression $$P_{opt} = I(\nu)\cdot \left(\frac{\Delta \nu}{\nu}\right)\cdot \nu \cdot  A \cdot \Omega$$ For an Antenna, the relation $A \Omega = \lambda^2$ always holds. Thus we can express the optical power in terms of the brightness temperature as $$P_{opt} = \eta_{opt} \cdot N_{pol} \left(\frac{\Delta \nu}{\nu}\right)\cdot \nu \cdot k_B T_{rj}$$ We replaced the factor of 2 with $N_{pol}$ to account for the actual number of polarizations being measured by the instrument. The optical efficiency accounts for the actual proportion of the incident power that is picked up by the resonator.

We start with the temperature of the bolometer, $T_b$. The power in the bolometer island, flows out into the thermal bath through the weak thermal link of the leg. The power in the leg, $P_{leg} = P_{opt} + P_{read}$, is the sum of the incident optical power and the readout power absorbed in the resonator bolometer. $P_{leg} = (1 + x) P_{opt}$. We model the heat flow through the leg as a power law i.e. $P_{leg} = K_{leg} \left(T_b^{\gamma_{leg}} - T_0^{\gamma_{leg}}\right)$. Rearranging this expression, we can solve for the temperature of the bolometer as 

$$T_b = \left[\frac{(1 + x)P_{opt}}{K_{leg}} + T_o^{\gamma_{leg}}\right]^{1/\gamma_{leg}}$$

With T_b calculated, we can now compute the long list of the resonator parameters below. These parameters are sensitive to the temperature because the quasiparticle density changes with the temperature.

1. $\omega_r = 1/\sqrt{L C}$ - resonant frequency of the resonator
2. $f_r = \omega_r/2 \pi$ - resonant frequency in Hz
3. $\Delta = 1.764\ k_B T_c$ - Electron energy gap width
4. $\eta = \frac{\hbar \omega}{2 k_B T_b}$ - useful quantity that appears over and over again
5. $S_1 (\omega) = \frac{2}{\pi}\sqrt{\frac{2 \Delta}{\pi k_B T_b}} \sinh\left(\eta(\omega) \right)\ K_0\left(\eta(\omega) \right)$ - real part of complex conductivity coefficient
6. $S_2 (\omega) = 1 + \sqrt{\frac{2 \Delta}{\pi k_B T_b}} \exp\left(-\eta(\omega) \right)\ I_0\left(\eta(\omega) \right)$ - imaginary part of complex conductivity coefficient
7. $\beta (\omega, T_b) = S_2(\omega)/ S_1 (\omega)$
8. $N_0 = \frac{3 \gamma}{2 \pi^2 k_B^2}$ - Density of states at the fermi surface
9. $Q_{qp} = (2 N_0 \Delta)/\left(\alpha \gamma_s S_1 (\omega) n_{qp}\right) $ - Internal quality factor from quasiparticle losses
10. $\Gamma_{gen}^{read} = \eta_{read} P_{read}/\Delta$ - Rate of qp generation by the readout power
11. $n_{th} = 2 N_0 \sqrt{2 \pi k_B T_b \Delta} \exp\left(-\frac{\Delta}{k_B T_b}\right)$ - thermally generated quasiparticle density
12. $\Gamma_{th} = \frac{n_{th} V_{sc}}{\tau_{max}}\left(1 +  \frac{n_{th}}{2 n_{qp}^*}\right)$ - Thermal generation rate of qps
12. $n_{qp} = \sqrt{\left(n_{th} + n_{qp}^*\right)^2 + 2 \Gamma_{gen}^{read} \tau_{max} n_{qp}^*/V_{sc}} - n_{qp}^*$ - quasiparticle number density
10. $Q_{\sigma} = \frac{\pi}{4} \exp\left(\Delta/k_B T_b\right)/\sinh\left(\eta(\omega) \right)\ K_0\left(\eta(\omega)\right) $ - Conductivity quality factor
11. $1/Q_i = 1/Q_{qp} + 1/Q_{int}$ - Total Internal Coupling Factor
12. $Q_c = Q_i$ - Assuming optimal coupling
13. $1/Q_r = 1/Q_i + 1/Q_c$ - resonator coupling constant
14. $\delta x = \frac{f_g - f_r}{f_r}$ - fractional resonator detuning
15. $S_{21} = 1 - \frac{Q_r}{Q_c} \frac{1}{1 + 2 j Q_r \delta x}$ - transmission parameter of the resonator
16. $\Delta f_r = f_r/2 Q_r$ - resonator bandwidth
17. $\Delta f_g = f_g - f_r$ - resonator detuning
18. $\phi = \arctan\left(\frac{\Delta f_g}{\Delta f_r}\right)$ - Phase angle of the resonator: $\phi = 0$ tuned, $\pi/2$ - perfectly detuned
19. $\chi_c \left(\rho_c = Q_c/Q_i\right) = \frac{4 \rho_c}{\left(1 + \rho_c \right)^2} = \frac{4 Q_r^2}{Q_i Q_c}$ - coupling efficiency
20. $\chi_g (\Delta f_g) = \left(1 + \left(\Delta f_g/ \Delta f_r \right)^2\right)^{-1}$ - detuning efficiency
21. $\chi_{qp} = Q_i/Q_{qp}$ - fraction of internal dissipation due to quasiparticles
22. $\tau_{th} = \tau_{max}/\left(1 + n_{th}/n_{qp}^*\right)$ - qp lifetime in equilibrium
23. $\tau_{qp} = \tau_{max}/\left(1 + n_{qp}/n_{qp}^*\right)$ - qp lifetime

In [4]:
omega_r = (1./np.sqrt(L*C)).to('1/s')
f_r = (omega_r/(2*np.pi)).to('Hz')
Delta = (1.764 * k_B * T_c).to('J')
P_opt = (eta_opt * N_pol * dnu_opt * nu_opt * k_B * T_rj).to('pW')
T_b = ((((1 + x)* P_opt)/K_leg + T_0**gamma_leg)**(1./gamma_leg)).to('K')
eta = (h * f_r / (2 * k_B * T_b)).value
S_1 = (2/np.pi)*np.sqrt(2*Delta/(np.pi*k_B*T_b))*np.sinh(eta * u.rad)*K_0(eta)
S_2 = 1 + np.sqrt(2*Delta/(np.pi*k_B*T_b)) * np.exp(-eta) * I_0(eta)
beta = S_1/S_2

N_0 = ((3 * (gamma * (rho/A_r)))/(2*np.pi**2 * k_B**2)).to('1/(J micrometer^3)')
Gamma_gen = (eta_read * x * P_opt/Delta).to('1/s')
n_th = (2*N_0 * np.sqrt(2*np.pi* k_B * T_b* Delta)*np.exp(-Delta/(k_B*T_b))).to('1/micrometer^3')
Gamma_th = ((n_th * V_sc/tau_max)* (1 + 0.5 * n_th/n_qp_star)).to('1/s')
n_qp = (np.sqrt((n_th + n_qp_star)**2 + (2*Gamma_gen*n_qp_star*tau_max/V_sc)) - n_qp_star).to('1/micrometer^3')
tau_th = (tau_max/(1 + n_th/n_qp_star)).to('us')
tau_qp = (tau_max/(1 + n_qp/n_qp_star)).to('us')
Q_qp = ((2 * N_0 * Delta)/(alpha * gamma_s * S_1 * n_qp)).to(1)
Q_sigma = (np.pi/4)*np.exp(Delta/(k_B * T_b))/np.sinh(eta)/K_0(eta)
Q_i = 1./(1/Q_qp + 1./Q_int)
Q_c = Q_i # For optimal performance
Q_r  = 1./(1./Q_c + 1./Q_i)

dx = ((f_g - f_r)/f_r).to(1)
S_21 = 1 - (Q_r/Q_c)* 1./(1 + 2 * 1j * Q_r * dx)
df_r = (f_r/(2*Q_r)).to('kHz')
df_g = (f_g - f_r).to('kHz')

phi = np.arctan2(df_g, df_r).to('degree')
chi_c = (4 * Q_r**2)/(Q_i * Q_c)
chi_g = 1./(1 + (df_g/df_r)**2)
chi_qp = Q_i/Q_qp

pprinter(omega_r=omega_r, f_r=f_r, Delta=Delta, P_opt=P_opt, T_b=T_b, eta=eta, tau_th=tau_th, tau_qp=tau_qp)
pprinter(S_1 = S_1, S_2 = S_2, N_0=N_0, Gamma_th=Gamma_th, Gamma_gen=Gamma_gen, n_th=n_th, n_qp=n_qp)
pprinter(Q_qp=Q_qp, Q_sigma=Q_sigma, Q_i=Q_i, Q_c=Q_c, Q_r=Q_r)
pprinter(dx=dx, S_21=S_21, df_r=df_r, df_g=df_g)
pprinter(phi=phi, chi_c=chi_c, chi_g=chi_g, chi_qp=chi_qp)

omega_r = 1.29e+09 1 / s
f_r = 2.05e+08 Hz
Delta = 2.92e-23 J
P_opt = 0.0233 pW
T_b = 0.307 K
eta = 0.016
tau_th = 70.2 us
tau_qp = 63.3 us
S_1 = 0.0908
S_2 = 3.06
N_0 = 1.08e+29 1 / (J um3)
Gamma_th = 4.11e+10 1 / s
Gamma_gen = 9.57e+09 1 / s
n_th = 6.13e+03 1 / um3
n_qp = 6.9e+03 1 / um3
Q_qp = 2.51e+04
Q_sigma = 1.13e+04
Q_i = 2.45e+04
Q_c = 2.45e+04
Q_r = 1.23e+04
dx = 7.05e-06
S_21 = 0.514+0.0839j
df_r = 8.36 kHz
df_g = 1.45 kHz
phi = 9.8 deg
chi_c =   1
chi_g = 0.971
chi_qp = 0.975


There are still a couple more useful parameters of the bolometer that we are interested in. First we can calculate the power from the generator, $P_g$ at the read out frequency from the expression $$P_{read} = \chi_{read} P_g$$, where $\chi_{read} = \frac{\chi_c \chi_g}{2} \leq \frac{1}{2}$ is the absorption efficiency. From this expression we see that at optimal coupling and zero detuning, half the generator power is absorbed in the resonator.

In [ ]:
chi_read = chi_c*chi_g/2
P_read = (x*P_opt).to('pW')
P_g = (P_read/chi_read).to('pW')